In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125463")
exp = Experiment(workspace=ws, name="quick-starts-ws-125463")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###cpu_cluster_name = "cpucluster"

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
Cpu_Cluster = ComputeTarget.create(ws, "CpuCluster", compute_config)

Cpu_Cluster.wait_for_completion(show_output=True)


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice

import os


# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C":uniform(0.05,1),
        "--max_iter":choice(20,40,60.80,100)
        
    
    }
) ### YOUR CODE HERE ###


# Specify a Policy
early_termination_policy = BanditPolicy(evaluation_interval=2,slack_factor=0.2) ### YOUR CODE HERE ###


# Create a SKLearn estimator for use with train.py
from azureml.core import ScriptRunConfig
from azureml.core import Environment

script='train.py'
est=SKLearn(source_directory='.',entry_script=script,compute_target=Cpu_Cluster)



# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=early_termination_policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=4,
                             max_concurrent_runs=4
                                    )
                              ### YOUR CODE HERE ###

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run=exp.submit(hyperdrive_config,show_output=True )
RunDetails(hyperdrive_run).show() #Run history widget
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print('Best Run Id: ', best_run.id)
print(best_run_metrics)

best_run.download_file("outputs/hyperdrive_model.joblib","./outputs/hyperdrive_model.joblib")
model=best_run.register_model(model_name='hyperdrive_model',model_path='outputs/hyperdrive_model.joblib')


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory


# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds=TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds) ### YOUR DATA OBJECT HERE ###

In [ ]:
#splitting the data
from sklearn.model_selection import train_test_split  
import pandas as pd
x_train,x_test,y_train,y_test= train_test_split(x,y, train_size=0.8, test_size=0.2, random_state=42) 
train_data=pd.concat((x_train,y_train),axis=1)


In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=2)

In [ ]:
#To fix Validation Error
!pip install -r /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.widgets import RunDetails
run_automl = exp.submit(automl_config, show_output=True)
RunDetails(run_automl).show()

In [ ]:
# Retrieve and save your best automl model.
### YOUR CODE HERE ###
best_run,fitted_automl_model=run_automl.get_output()
print(best_run)
print(fitted_automl_model)
from azureml.core import Model
best_run.register_model(model_path="outputs/model.joblib",model_name="automl_model")
#joblib.dump(fitted_automl_model,'automl_model.joblib')
#automl_model=run.register_model(model_name='automl_model',outputpath='outputs/model_automl.joblib')
